<a href="https://colab.research.google.com/github/iskra3138/ImageSr/blob/master/3_DataSet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x

import tensorflow as tf

TensorFlow 2.x selected.


In [0]:
import numpy as np

### TFRecord 활용하여 DataSet 만들기

In [0]:
CLASSES = ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']

In [0]:
import os

# TFRecord Parsing을 위한 함수 정의  
AUTO = tf.data.experimental.AUTOTUNE

IMG_WIDTH = 224
IMG_HEIGHT = 224
IMAGE_SIZE =  [IMG_HEIGHT, IMG_WIDTH]

batch_size = 64

## 본 실험에서는 16개의 tfrecord파일을 train/validation용으로 나눠서 사용합니다.
## train전용, validation전용 tfrecord 파일들이 있으면 특정해서 list 로 넘기시면 됩니다.
BUCKET = "gs://iskra3138_share"
gcs_pattern = os.path.join(BUCKET, '*.tfrec')
validation_split = 0.19
filenames = tf.io.gfile.glob(gcs_pattern)
split = len(filenames) - int(len(filenames) * validation_split)
train_fns = filenames[:split]
validation_fns = filenames[split:]

## TFRecord Parsing 함수 (TFRecord 생성함수를 참고해서 만들어줍니다.)
def parse_tfrecord(example):
    features = {
        "image": tf.io.FixedLenFeature([], tf.string),  # tf.string = bytestring (not text string)
        "file_name": tf.io.FixedLenFeature([], tf.string),  # one bytestring
        "label_name": tf.io.FixedLenFeature([], tf.string),  # one bytestring
        "label": tf.io.FixedLenFeature([], tf.int64),  # shape [] means scalar, one integer
    }
    # decode the TFRecord
    example = tf.io.parse_single_example(example, features)
    
    
    label = example['label']
    label = tf.one_hot(indices=label, depth=5)   
    image = tf.image.decode_jpeg(example['image'], channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32) ## make [0,255] to [0,1) resize 앞에 위치할 때만 [0,1), 즉 input이 float32가 아니어야 작동
    image = tf.image.resize(image, IMAGE_SIZE) ## method가 tf.image.ResizeMethod.NEAREST_NEIGHBOR 가 아니면 출력은 무조건 float32
    
    file_name  = example['file_name']
    label_name  = example['label_name']
    
    return image, label, file_name, label_name

def load_dataset(filenames):
  # Read from TFRecords. For optimal performance, we interleave reads from multiple files.
  records = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO)
  return records.map(parse_tfrecord, num_parallel_calls=AUTO)

def get_training_dataset():
  dataset = load_dataset(train_fns)

  # Create some additional training images by randomly flipping and
  # increasing/decreasing the saturation of images in the training set. 
  def data_augment(image, label, file_name, label_name):
    modified = tf.image.random_flip_left_right(image)
    modified = tf.image.random_flip_up_down(modified)
    return modified, label, file_name, label_name
  augmented = dataset.map(data_augment, num_parallel_calls=AUTO)

  # Prefetch the next batch while training (autotune prefetch buffer size).
  return augmented.repeat().shuffle(2048).batch(batch_size).prefetch(AUTO) 

training_dataset = get_training_dataset()
validation_dataset = load_dataset(validation_fns).batch(batch_size).prefetch(AUTO)

위 코드를 실행시키면, training_dataset, validation_dataset 모두 (image, label, file_name, label_name)구조로 element들이 구성됨

training_dataset은 
- augmented = dataset.map(data_augment, num_parallel_calls=AUTO)에서 랜덤으로 좌우플립/상하플립이 적용되고, 
- augmented.repeat().shuffle(2048).batch(batch_size).prefetch(AUTO) 에서 무한 반복으로, 섞고, batch_size만큼 생성됨

validation_dataset은 
- batch_size만큼 뽑히는 데, 전체 element들이 다 뽑히면 끝남

### TFrecord 파일 속에 있는 데이터들 개수 세기

In [5]:
## tfrecord 파일 내 데이터 개수 세기
#### tfrecord 이름이 아니라 직접 데이터 개수를 확인하고 싶다면 아래 코드 실행
#sum([1 for _ in tf.data.TFRecordDataset({TFRecord 파일 이름})])
sum([1 for _ in tf.data.TFRecordDataset(filenames[0])])

230

In [6]:
## tfrecord 파일 내 데이터 개수 세기
#### tfrecord 이름이 아니라 직접 데이터 개수를 확인하고 싶다면 아래 코드 실행
#### 여러 개의 tfrecord를 list로 넘겨도 됨

#sum([1 for _ in tf.data.TFRecordDataset([TFRecord 파일 리스트])])
sum([1 for _ in tf.data.TFRecordDataset(filenames[0:3])])

690

In [7]:
## train_fns list로 traind_data 개수 세기
print (train_fns)
print (sum([1 for _ in tf.data.TFRecordDataset(train_fns)]))

['gs://iskra3138_share/00-230.tfrec', 'gs://iskra3138_share/01-230.tfrec', 'gs://iskra3138_share/02-230.tfrec', 'gs://iskra3138_share/03-230.tfrec', 'gs://iskra3138_share/04-230.tfrec', 'gs://iskra3138_share/05-230.tfrec', 'gs://iskra3138_share/06-230.tfrec', 'gs://iskra3138_share/07-230.tfrec', 'gs://iskra3138_share/08-230.tfrec', 'gs://iskra3138_share/09-230.tfrec', 'gs://iskra3138_share/10-230.tfrec', 'gs://iskra3138_share/11-230.tfrec', 'gs://iskra3138_share/12-230.tfrec']
2990


In [8]:
## validation_fns list로 traind_data 개수 세기
print (validation_fns)
print (sum([1 for _ in tf.data.TFRecordDataset(validation_fns)]))

['gs://iskra3138_share/13-230.tfrec', 'gs://iskra3138_share/14-230.tfrec', 'gs://iskra3138_share/15-220.tfrec']
680


train data는 2990장, validation data는 680장임

### 1batch 데이터 n 개 추출하기

In [9]:
## dataset 랜덤으로 1batch 데이터 뽑기 1
## as_numpy_iterator()를 써서 next로 뽑기
iter = validation_dataset.as_numpy_iterator()
img, label, file_name, label_name = next(iter)
label1 = label
print (label1.shape)

(64, 5)


In [10]:
# as_numpy_iterator()의 출력을 numpy array
type(label1)

numpy.ndarray

In [13]:
## dataset 랜덤으로 1batch 데이터 뽑기 2
### dataset 자체를 쓰되, break
### break를 쓰지 않으면 as_numpy_iterator()를 for문으로 돌릴때처럼 전체 데이터셋에 대해 한 바퀴 돔
for img, label, file_name, label_name in validation_dataset :
  label2 = label
  break
print (label2.shape)

(64, 5)


In [17]:
# take()의 출력을 EagerTensor
type(label2)

tensorflow.python.framework.ops.EagerTensor

In [16]:
## dataset 랜덤으로 1batch 데이터 뽑기 2
### take() 사용
### take(n)을 쓰면 사전에 정의된 배치사이즈 크기의 데이터를 n번 샘플링 해옴
for img, label, file_name, label_name in validation_dataset.take(1):
  label3 = label
  print (label3.shape)

(64, 5)


In [15]:
# take()의 출력을 EagerTensor
type(label3)

tensorflow.python.framework.ops.EagerTensor

In [0]:
# 순서가 명시된 Dataset 정의
# (0,10), (1, 11), ..., (9, 19) 형태의 DataSet을 만듦
a = np.arange(10)
b = np.arange(10, 20)
print (a)
print (b)
range_dataset = tf.data.Dataset.from_tensor_slices((a, b))

[0 1 2 3 4 5 6 7 8 9]
[10 11 12 13 14 15 16 17 18 19]


In [0]:
## dataset 랜덤으로 1batch 데이터 뽑기 1
## as_numpy_iterator()를 써서 next로 뽑기
iter = range_dataset.as_numpy_iterator()
for i in range(10):
  data = next(iter)
  print (data)

(0, 10)
(1, 11)
(2, 12)
(3, 13)
(4, 14)
(5, 15)
(6, 16)
(7, 17)
(8, 18)
(9, 19)


In [0]:
## dataset 랜덤으로 1batch 데이터 뽑기 2
### take() 사용
### 출력값은 EagerTensor의 Tuple이므로 출력 결과 중 numpy= 다음의 값을 보면 됨
for data in range_dataset.take(1):
  print (data)

(<tf.Tensor: shape=(), dtype=int64, numpy=0>, <tf.Tensor: shape=(), dtype=int64, numpy=10>)


In [0]:
## dataset 랜덤으로 1batch 데이터 뽑기 2
### take() 사용시 numpy 값을 뽑고 싶다면 요소별로 변수로 받은 다음 numpy() 하면 됨

for data1, data2 in range_dataset.take(10):
  print (data1.numpy(), data2.numpy())

0 10
1 11
2 12
3 13
4 14
5 15
6 16
7 17
8 18
9 19


In [0]:
## as_numpy_iterator()를 정의해서 사용할 때 1epoch이 넘어가서도 호출하면 error 발생
iter = range_dataset.as_numpy_iterator()
for i in range(11): ## data의 개수 10개를 넘어가면 error 발생
  data = next(iter)
  print (data)

(0, 10)
(1, 11)
(2, 12)
(3, 13)
(4, 14)
(5, 15)
(6, 16)
(7, 17)
(8, 18)
(9, 19)


In [0]:
## take()를 사용하면 10개를 넘어가도 작동함, 10개에서 작동이 정지

for data1, data2 in range_dataset.take(20):
  print (data1.numpy(), data2.numpy())

0 10
1 11
2 12
3 13
4 14
5 15
6 16
7 17
8 18
9 19


###  DataSet 내에 있는 모든 데이터를 한번씩만 사용하기

In [0]:
# TFRecord Data Set으로 재실험
## as_numpy_iterator()를 쓰면 딱 1epoch만큼 샘플링할 수 있음
## 64 batch씩 10번 남은 40batch 1번, 총 11번 반복함
num_samples = 0
for img, label, file_name, label_name in validation_dataset.as_numpy_iterator():
  num_samples += label.shape[0]
print (num_samples)

680


전체 데이터 개수와 똑같아졌음

In [0]:
## take(n)를 쓰면 계속해서 증가하다가 1epoch 샘플링에서 끝남
for n in range(15) :
  num_samples = 0
  for img, label, file_name, label_name in validation_dataset.take(n):
    num_samples += label.shape[0]
  print (n, num_samples)

0 0
1 64
2 128
3 192
4 256
5 320
6 384
7 448
8 512
9 576
10 640
11 680
12 680
13 680
14 680


계속해서 64개씩 늘어나다 11번째는 남은 40만큼 늘어나고 그 이후는 변화가 없음

### 여러 Class가 섞여 있는 DataSet에서 Class 별 DataSet 만들기

In [0]:
# class별 데이터세트 만들기
## 원하는 클래스끼리만 모아 놓고 싶을 때는 fliter를 적용하면 됨
### label이 1인 데이터세트를 만드는 예제

def select_label1 (img, label, file_name, label_name):
  return tf.math.equal(tf.argmax(label), 1) ## label이 one-hot 형태임

label1dataset = validation_dataset.unbatch().filter(select_label1) ## 한 장씩 평가해야 해서 unbatch 해줌

In [0]:
for img, label, file_name, label_name in label1dataset.as_numpy_iterator():
  print (label)

[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0

In [0]:
label.numpy().shape

(64, 5)